In [28]:
import pandas as pd
import numpy as np
from datetime import timedelta 
import matplotlib.pyplot as plt
import statsmodels as sm
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [2]:
df1 = pd.read_csv('../../data/train_validation_marker.csv')

In [3]:
df2 = df1[df1['Encoded_SKU_ID'].isin([27, 28, 65, 83, 203, 223, 240, 288, 308, 418, 439, 451, 478])]

In [7]:
df3 = df1[df1['validation_clean'] == True]

In [8]:
df3.head()

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS,max_SALES_DATE,delta_days,validation,validation_clean
0,1,2017-09-28,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,?,Fully-Stocked,1,2022-07-31,1767,False,True
1,2,2017-11-24,CAMERA BATTERIES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,39.99,?,?,Fully-Stocked,18,2022-07-31,1710,False,True
2,3,2019-11-28,SO PARTS,MOBILE ACCESSORIES,ML - Connected Car,VP - Electrify & Car,49.99,?,46.98,Constrained,7,2022-07-31,976,False,True
4,1,2018-10-27,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,?,Fully-Stocked,7,2022-07-31,1373,False,True
5,5,2018-12-18,SO CAR ACCESSORIES,MOBILE ACCESSORIES,ML - Connected Car,VP - Electrify & Car,16.99,?,?,Fully-Stocked,5,2022-07-31,1321,False,True


In [10]:
df3.Encoded_SKU_ID.nunique()

545

In [13]:
rst_sku_df = pd.read_excel('../../results/rmse_all_preds_jan_17.xlsx')

In [14]:
rst_sku_df.Encoded_SKU_ID

,Encoded_SKU_ID,rmse_du_15,mean_du_15,pct_rmse_15,rmse_du_30,mean_du_30,pct_rmse_30,rmse_du_45,mean_du_45,pct_rmse_45,...,pct_rmse_prophet,rmse_du_null,mean_du_null,pct_rmse_null,rmse_du_holt,mean_du_holt,pct_rmse_holt,min_rmse,argmin_period,min_period
0,82,4.722325,0.428571,11.018759,4.840344,0.428571,11.294136,1.102108,0.428571,2.571586,...,5.362264,3.605551,0.428571,8.412953,9.753826,0.428571,22.758928,0.911043,5,rmse_du_max
1,568,4.097124,4.285714,0.955996,3.447670,4.285714,0.804456,3.652836,4.285714,0.852329,...,0.595404,5.070926,4.285714,1.183216,3.024439,4.285714,0.705702,2.551733,8,rmse_du_prophet
2,317,3.766488,0.857143,4.394235,1.362770,0.857143,1.589899,1.784957,0.857143,2.082450,...,2.297158,1.069045,0.857143,1.247219,2.363641,0.857143,2.757581,1.069045,9,rmse_du_null
3,95,3.668447,1.285714,2.853236,1.698949,1.285714,1.321405,3.991151,1.285714,3.104228,...,1.743961,1.309307,1.285714,1.018350,1.720612,1.285714,1.338254,1.309307,9,rmse_du_null
4,340,3.668447,0.714286,5.135825,2.077172,0.714286,2.908041,2.329929,0.714286,3.261901,...,0.970033,0.925820,0.714286,1.296148,1.092959,0.714286,1.530142,0.692881,8,rmse_du_prophet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,558,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,...,11.498812,1.069045,1.000000,1.069045,19.176286,1.000000,19.176286,1.069045,0,rmse_du_15
541,559,NaN,3.857143,NaN,NaN,3.857143,NaN,NaN,3.857143,NaN,...,1.017409,3.207135,3.857143,0.831479,4.635827,3.857143,1.201881,3.207135,0,rmse_du_15
542,560,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,...,NaN,0.534522,1.000000,0.534522,0.827778,1.000000,0.827778,0.534522,0,rmse_du_15
543,566,NaN,5.571429,NaN,NaN,5.571429,NaN,NaN,5.571429,NaN,...,NaN,2.507133,5.571429,0.449998,15.125061,5.571429,2.714755,2.507133,0,rmse_du_15


In [19]:
hw_sku = rst_sku_df[rst_sku_df['min_period']=='rmse_du_holt']

In [151]:
sku = list(hw_sku.Encoded_SKU_ID)+[76, 147, 157, 358, 413, 485, 569]

In [51]:
len(sku)

18

In [24]:
test = pd.read_excel('../../data/Validation_Data.xlsx')

In [37]:
test.Encoded_SKU_ID.nunique()

539

In [150]:
set(test.Encoded_SKU_ID) - set(rst_sku_df.Encoded_SKU_ID)

{76, 147, 157, 358, 413, 485, 569}

In [43]:
test[test.Encoded_SKU_ID.isin([27, 28, 65, 83, 203, 223, 240, 288, 308, 418, 439, 451, 478])]

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS,Forecasted Units


In [87]:
def fit_es(sku,df1,test,seasonal_periods=62):
    y_pred = pd.DataFrame()
    y_actual = pd.DataFrame()

    for i in sku:
        print( i)
    # clean train data on item i
        train = df1[df1['Encoded_SKU_ID']==i][['Encoded_SKU_ID','SALES_DATE','DAILY_UNITS']]
        train['SALES_DATE'] = pd.to_datetime(train['SALES_DATE'])
        train.set_index('SALES_DATE', inplace = True)
    
        val = test[test['Encoded_SKU_ID']==i][['Encoded_SKU_ID','SALES_DATE','DAILY_UNITS']]
        val['SALES_DATE'] = pd.to_datetime(val['SALES_DATE'])
        val.set_index('SALES_DATE', inplace = True)
    
    # train the HW model with auto train & make the forecast
        model = ExponentialSmoothing(train['DAILY_UNITS'],trend='add',seasonal='add',seasonal_periods=seasonal_periods).fit()
        forecast = model.forecast(7)
        print(forecast)
    
    #Construct the predicted dataframe
        SALES_DATE = pd.date_range(start='08/01/2022', end='08/07/2022')
        prediction = pd.DataFrame(forecast,columns=['predicted']).reset_index(drop=True).set_index(SALES_DATE)
        print(prediction.index)
        prediction['Encoded_SKU_ID'] = val['Encoded_SKU_ID']
        prediction.reset_index(inplace = True)
        y_pred = pd.concat([y_pred, prediction], axis = 0)
    
    #Construct the actual dataframe
        val.reset_index(inplace = True)
        val.rename(columns={'DAILY_UNITS':'actual'}, inplace = True)
        y_actual = pd.concat([y_actual, val], axis = 0)
        
        
    return(y_pred,y_actual)

In [152]:
holt_rst, holt_actual = fit_es(sku, df1, test)

365


/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has n

1218    1.598060
1219    1.334868
1220    1.755911
1221    1.440121
1222    1.492759
1223    1.387498
1224    1.913823
dtype: float64
DatetimeIndex(['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04',
               '2022-08-05', '2022-08-06', '2022-08-07'],
              dtype='datetime64[ns]', freq='D')
190


/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


1891    1.670673
1892    1.836632
1893    2.035384
1894    1.537414
1895    1.734490
1896    1.536760
1897    1.701242
dtype: float64
DatetimeIndex(['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04',
               '2022-08-05', '2022-08-06', '2022-08-07'],
              dtype='datetime64[ns]', freq='D')
248


...

/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


1276    16.560530
1277    19.797115
1278    30.818005
1279    13.349868
1280    21.696092
1281    22.102015
1282    18.817382
dtype: float64
DatetimeIndex(['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04',
               '2022-08-05', '2022-08-06', '2022-08-07'],
              dtype='datetime64[ns]', freq='D')
413


/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction 

937    3.603303
938    3.039835
939    3.959853
940    5.141440
941    4.304999
942    3.448650
943    2.580019
dtype: float64
DatetimeIndex(['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04',
               '2022-08-05', '2022-08-06', '2022-08-07'],
              dtype='datetime64[ns]', freq='D')
485
1233    0.908757
1234    0.700219
1235    0.997280
1236    1.447328
1237    1.409739
1238    1.418128
1239    1.678635
dtype: float64
DatetimeIndex(['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04',
               '2022-08-05', '2022-08-06', '2022-08-07'],
              dtype='datetime64[ns]', freq='D')
569
701    2.883637
702    2.065078
703    1.519381
704    2.246047
705    1.791683
706    1.430173
707    1.793841
dtype: float64
DatetimeIndex(['2022-08-01', '2022-08-02', '2022-08-03', '2022-08-04',
               '2022-08-05', '2022-08-06', '2022-08-07'],
              dtype='datetime64[ns]', freq='D')


/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yibeihu/opt/anaconda3/envs/bestbuy_project/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [153]:
holt_rst.rename({'index':'SALES_DATE'},axis=1, inplace= True)

In [155]:
holt_rst.shape

(175, 3)

In [156]:
175/7

25.0

In [33]:
import sys
sys.path.append("/Users/yibeihu/bestbuy_project/")
from src.utils import *

In [157]:
rmse(holt_rst, holt_actual)

10.306690889821096

In [158]:
holt_rst.to_csv('../../results/holt_rst_final.csv')

In [58]:
df1[(df1['Encoded_SKU_ID']==1) & (df1['SALES_DATE']=='2022-07-31')]

,Encoded_SKU_ID,SALES_DATE,SUBCLASS_NAME,CLASS_NAME,ML_NAME,CATEGORY_NAME,RETAIL_PRICE,PROMO_PRICE,COMPETITOR_PRICE,Inventory,DAILY_UNITS,max_SALES_DATE,delta_days,validation,validation_clean
411277,1,2022-07-31,MISC CAMERA ACCES,DIGITAL CAMERA ACCY,ML - DI Accessories,VP - Digital Imaging,89.99,?,69.95,Fully-Stocked,1,2022-07-31,0,True,True


In [120]:
y_actual = test[['Encoded_SKU_ID','SALES_DATE', 'DAILY_UNITS']]

In [122]:
y_actual.shape

(3773, 3)

In [124]:
forecast = df1[df1['SALES_DATE'] == '2022-07-31'][['Encoded_SKU_ID','SALES_DATE','DAILY_UNITS']]

In [125]:
forecast.shape

(556, 3)

In [126]:
y_pred = y_actual.merge(forecast, on=['Encoded_SKU_ID'], how='left')

In [127]:
y_pred.head()

,Encoded_SKU_ID,SALES_DATE_x,DAILY_UNITS_x,SALES_DATE_y,DAILY_UNITS_y
0,23,2022-08-06,3,2022-07-31,1
1,100,2022-08-02,2,2022-07-31,2
2,121,2022-08-06,6,2022-07-31,0
3,207,2022-08-03,0,2022-07-31,1
4,416,2022-08-05,9,2022-07-31,11


In [129]:
y_pred.isna().sum()

Encoded_SKU_ID    0
SALES_DATE_x      0
DAILY_UNITS_x     0
SALES_DATE_y      0
DAILY_UNITS_y     0
dtype: int64

In [130]:
y_pred_null = y_pred[['Encoded_SKU_ID','SALES_DATE_x','DAILY_UNITS_y']].rename(columns={'SALES_DATE_x':'SALES_DATE','DAILY_UNITS_y':'predicted'})

In [140]:
y_pred_null.to

,Encoded_SKU_ID,SALES_DATE,predicted
0,23,2022-08-06,1
1,100,2022-08-02,2
2,121,2022-08-06,0
3,207,2022-08-03,1
4,416,2022-08-05,11
...,...,...,...
3768,486,2022-08-06,5
3769,171,2022-08-02,0
3770,258,2022-08-01,0
3771,140,2022-08-01,2


In [144]:
y_act_null = y_actual.rename(columns={'DAILY_UNITS':'actual'})

In [145]:
y_act_null

,Encoded_SKU_ID,SALES_DATE,actual
0,23,2022-08-06,3
1,100,2022-08-02,2
2,121,2022-08-06,6
3,207,2022-08-03,0
4,416,2022-08-05,9
...,...,...,...
3768,486,2022-08-06,2
3769,171,2022-08-02,1
3770,258,2022-08-01,0
3771,140,2022-08-01,1


In [148]:
rmse(y_pred_null,y_act_null)

5.290976671399268

In [149]:
y_pred_null.to_csv('../../results/null_model_results_final.csv')